In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from scipy.optimize import minimize

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
data_df = pd.read_excel(r"china_cobb_douglas_data.xlsx")

In [ ]:
data_df = data_df.T

In [ ]:
data_df.columns = data_df.iloc[2]

In [ ]:
data_df = data_df.reset_index(drop=False)

In [ ]:
data_df.columns.values[0] = 'report_year'

In [ ]:
data_df = data_df[5:-1]

In [ ]:
data_df = data_df[data_df['report_year'].between(1978,2023)]

# USD

In [ ]:
# making a df with the correct currency capital stock and GDP
CHINA_current_usd = data_df[['report_year','Capital stock (current US$)', 'Labor force, total', 'GDP (current US$)']].reset_index(drop=True)

## 1991-2022

In [ ]:
# seperating them into the dependant (GDP) and independant (cap stock and labor) variables
X = CHINA_current_usd[CHINA_current_usd.columns[1:-1]]
y = CHINA_current_usd[CHINA_current_usd.columns[-1]]

In [ ]:
# taking the logs of the data so that the elasticity of the two independant variables would be their coefficicents
X[X.columns[0]] = np.log(X[X.columns[0]].astype('float64'))
X[X.columns[1]] = np.log(X[X.columns[1]].astype('float64'))

# in a linearised version of cobb douglass (by taking the log of the equation) we get the log of TFP as a constant, in order to find it, we need to explicitly insert it
X['TFP'] = 1

# taking the log of the dependant variable (GDP) as per the lin version of cobb douglass
y = np.log(y.astype('float64'))

In [ ]:
# defining the MSE Function to minimize
def mse_loss(weights, X, y):
    # x_, x0, x1, x2 = x
    y_pred = X.dot(weights)
    return np.mean((y_pred - y) ** 2)

# initial guess (all ones cause why not, it dont really matter)
initial_guess = np.ones(len(X.T))*0.5

# bounds (common bounds, elasticity and TFP are positive, but the last one doesnt have an upper bound 
bounds = [(0, 1), (0, 1), (0, np.inf)]

# constraints (assuming constant return to scale, thus the ealsticities must sum to 1)
constraints = [
    {'type': 'eq'
     , 'fun': lambda weights: weights[0] + weights[1] - 1       # x1 + x2 = 1 (equality: x1 + x2 - 1 = 0)
    }
]

In [ ]:
# minimizing the mse function
result = minimize(
    mse_loss,
    initial_guess,
    args=(X, y),
    method='trust-constr',
    bounds=bounds,
    constraints=constraints,
    options={'disp': True
             , 'maxiter': 15000}
)

In [ ]:
cobb_douglass_1978_2023_df = pd.concat([pd.DataFrame(X.columns).T, pd.DataFrame(result.x.round(4)).T]).T
cobb_douglass_1978_2023_df.to_excel(r'cobb_douglass_1978_2023.xlsx', index=False)

In [ ]:
cd_wellness_fit_1978_2023_df = CHINA_current_usd.copy()
cd_wellness_fit_1978_2023_df['GDP (current US$)_model'] = cd_wellness_fit_1978_2023_df['Capital stock (current US$)'] ** result.x[0] * cd_wellness_fit_1978_2023_df['Labor force, total'] ** result.x[1] * np.e**result.x[2]
cd_wellness_fit_1978_2023_df.to_excel(r'cd_wellness_fit_1978_2023.xlsx', index=False)

## 1991-2022 (5 year intervals)

In [ ]:
# seperating them into the dependant (GDP) and independant (cap stock and labor) variables
X = CHINA_current_usd[CHINA_current_usd.columns[1:-1]]
y = CHINA_current_usd[CHINA_current_usd.columns[-1]]

# taking the logs of the data so that the elasticity of the two independant variables would be their coefficicents
X[X.columns[0]] = np.log(X[X.columns[0]].astype('float64'))
X[X.columns[1]] = np.log(X[X.columns[1]].astype('float64'))

# in a linearised version of cobb douglass (by taking the log of the equation) we get the log of TFP as a constant, in order to find it, we need to explicitly insert it
X['TFP'] = 1

# taking the log of the dependant variable (GDP) as per the lin version of cobb douglass
y = np.log(y.astype('float64'))

In [ ]:
results = []
window_size = 5

for i in range(len(CHINA_current_usd['report_year']) - window_size + 1):
    # Extract the current window
    X_window = X[i : i + window_size]
    y_window = y[i : i + window_size]

    # defining the MSE Function to minimize
    def mse_loss_window(weights, X_window, y_window):
        # x_, x0, x1, x2 = x
        y_pred = X_window.dot(weights)
        return np.mean((y_pred - y_window) ** 2)

    # initial guess (all ones cause why not, it dont really matter)
    if i == min(range(len(CHINA_current_usd['report_year']) - window_size + 1)):
        initial_guess = np.ones(len(X.T))*0.5
    
    # bounds (common bounds, elasticity and TFP are positive, but the last one doesnt have an upper bound 
    bounds = [(0, 1), (0, 1), (0, np.inf)]

    # constraints (assuming constant return to scale, thus the ealsticities must sum to 1)
    constraints = [
        {'type': 'eq'
         , 'fun': lambda weights: weights[0] + weights[1] - 1       # x1 + x2 = 1 (equality: x1 + x2 - 1 = 0)
        }
    ]
    
    # minimizing the mse function
    result = minimize(
        mse_loss_window,
        initial_guess,
        args=(X_window, y_window),
        method='trust-constr',
        bounds=bounds,
        constraints=constraints,
        options={'disp': True
                 , 'maxiter': 15000
                }
    )

    initial_guess = result.x

    # Store results
    results.append({
        'start_year': CHINA_current_usd['report_year'][i],
        'end_year': CHINA_current_usd['report_year'][i + window_size - 1],
        'capital_elast': result.x[0],
        'labor_elast': result.x[1],
        'TFP': result.x[2],
        'mse': result.fun
    })

results_df = pd.DataFrame(results)

In [ ]:
results_df.to_excel(r'cobb_douglass_5_year_estimations_1978_2023.xlsx', index=False)

In [ ]:
actual_model_GDP = CHINA_current_usd[['report_year','GDP (current US$)']]

for i in range(len(CHINA_current_usd['report_year']) - window_size + 1):

    start_year = CHINA_current_usd['report_year'][i]
    end_year = CHINA_current_usd['report_year'][i + window_size - 1]
    
    five_year_sample_df = CHINA_current_usd[CHINA_current_usd['report_year'].between(start_year, end_year)]
    cobb_douglass = results_df[results_df['start_year']==CHINA_current_usd['report_year'][i]].reset_index(drop=True)
    five_year_sample_df[f'''GDP (current US$)_model_{end_year}'''] = five_year_sample_df['Capital stock (current US$)'] ** cobb_douglass['capital_elast'][0] * five_year_sample_df['Labor force, total'] ** cobb_douglass['labor_elast'][0] * np.e**cobb_douglass['TFP'][0]
    
    actual_model_GDP = pd.merge(actual_model_GDP, five_year_sample_df[['report_year',f'''GDP (current US$)_model_{end_year}''']], how='left', on=['report_year'])

actual_model_GDP['GDP (current US$)_model_avg'] = actual_model_GDP[actual_model_GDP.columns[2:]].T.mean()

In [ ]:
actual_model_GDP.to_excel(r'cd_5_year_estimations_wellness_fit_1978_2023.xlsx', index=False)

In [ ]:
# np.e**results_df[results_df['start_year']==min(results_df['start_year'])].TFP[0]

In [ ]:
# np.e**results_df[results_df['start_year']==max(results_df['start_year'])].TFP[41]

In [ ]:
# np.e**results_df[results_df['start_year']==max(results_df['start_year'])].TFP[41]/np.e**results_df[results_df['start_year']==min(results_df['start_year'])].TFP[0]

In [ ]:
n = 2010#max(results_df['start_year'])
# n = min(results_df['start_year'])

In [ ]:
test_df = CHINA_current_usd[CHINA_current_usd['report_year'].between(n, n+window_size-1)]
test_df

In [ ]:
cdm = results_df[results_df['start_year']==n].reset_index(drop=True)
cdm

In [ ]:
test_df['GDP (current US$)_model'] = test_df['Capital stock (current US$)'] ** cdm['capital_elast'][0] * test_df['Labor force, total'] ** cdm['labor_elast'][0] * np.e**cdm['TFP'][0]

In [ ]:
test_df['GDP (current US$)_model']/test_df['GDP (current US$)']